# Laboratório 01 de IoT com Yolo e InfluxDB - Guia para a aula experimental

Vamos iniciar nosso notebook instalando e posteriormente chamando todas as bibliotecas que usaremos. Nessa etapa, nada precisa ser modificado.

In [ ]:
!pip install arrow influxdb_client

In [ ]:
import numpy as np
import pandas as pd
import arrow
import requests
import re
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import matplotlib.pyplot as plt

O próximo passo é definir os endereços que serão usados para consultar e enviar os dados, o token de acesso, a organização e o bucket com os dados.

In [ ]:
token = ''
user = ''
url = ''
bucket = ''

Vamos agora conectar com o banco de dados, usando as np.infomções definidas anteriormente.

In [ ]:
db_client = influxdb_client.InfluxDBClient(url=url, token=token, org=user)

## Vamos baixar os dados e ver como eles se comportam
Para iniciar esse trabalho, vamos primeiro escrever uma função que nos permita baixar os dados dos útimos "d" dias, do bucket que definimos anteriormete.

In [ ]:
def get_data(days):
    query_api = db_client.query_api()
    stop = arrow.now().to('UTC').isoformat()[:-13]+'Z'
    start = arrow.now().shift(days=-1*days).to('UTC').isoformat()[:-13]+'Z'
    query = """from(bucket: \""""+bucket+"""\")
    |> range(start: """+start+""", stop: """+stop+""")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    |> keep(columns: ["_time", "_measurement","quantidade", "model"])"""
    df = query_api.query_data_frame(query, org=user)
    return df

Agora vamos usar a funçar que definimos acima e baixar o último dia de dados. Vamos aproveitar para deixar o timestamp no horário do Brasil e organizar o Dataframe pelo tempo.

In [ ]:
df = get_data(1)
df['_time'] = df['_time'].dt.tz_convert('America/Sao_Paulo')
df.index = pd.to_datetime(df['_time'])
df = df.drop(columns=['_time'])

Caso tudo tenha funcionado como esperado, você deve estar vendo seus dados na sequência, já no formato tabular do Pandas.

In [ ]:
df

Por último vamos Fazer o gráfico da quantidade de pessoas detectadas ao longo do tempo

In [ ]:
plt.figure(figsize=(15,4))
for model in df['model'].unique():
    model_data = df[df['model'] == model]
    plt.plot(model_data.index, model_data['quantidade'], label=model)
plt.xticks(rotation=45);
plt.title("Serie temporal de cada modelo")
plt.xlabel("Tempo")
plt.ylabel("Quantidade")
plt.legend(title="Modelos")
plt.grid(True)